In [1]:
# Import libraries 
# https://pub.towardsai.net/emoticon-and-emoji-in-text-mining-7392c49f596a
# https://medium.com/geekculture/text-preprocessing-how-to-handle-emoji-emoticon-641bbfa6e9e7
from emot.emo_unicode import EMOTICONS_EMO, EMOJI_UNICODE 
import pandas as pd
import re

# Wrangle Anime Data

In [ ]:
# Load data
# https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews?select=animes.csv
animes = pd.read_csv("data/animes.csv") 
# https://www.kaggle.com/datasets/azathoth42/myanimelist?select=AnimeList.csv
AnimeList = pd.read_csv("data/AnimeList.csv")

In [ ]:
AnimeList = AnimeList[["anime_id", "type", "source", "producer", "licensor", "studio"]]
animes = animes.merge(AnimeList, how = "left", left_on = "uid", right_on = "anime_id")
anime = animes[["uid", 
                "title", 
                "synopsis", 
                "genre", 
                "type",
                "episodes", 
                "source", 
                "producer", 
                "licensor", 
                "studio"]]
anime = anime.drop_duplicates()
anime.head()

# Wrangle Reviews Data

In [ ]:
# reviews = pd.read_csv("data/reviews.csv")
# reviews = reviews.drop_duplicates()

In [ ]:
# import math

# total_rows = len(reviews)
# max_rows = 10000
# num_files = math.ceil(total_rows/max_rows)

# start = 0
# end = 9999

# for i in range(1, num_files + 1):
#     i = str(i)
#     print("Writing file #" + i)
#     reviews.iloc[start:(end + 1), :].to_csv("data/reviews/reviews" + i + ".csv", index = False)
#     start += 10000
#     end += 10000

In [2]:
# Read in data
reviews = pd.read_csv("data/reviews/reviews1.csv")
for i in range(2, 15):
    i = str(i)
    print("Concatenating data file #" + i)
    addition = pd.read_csv("data/reviews/reviews" + i + ".csv")
    reviews = pd.concat([reviews, addition])

Concatenating data file #2
Concatenating data file #3
Concatenating data file #4
Concatenating data file #5
Concatenating data file #6
Concatenating data file #7
Concatenating data file #8
Concatenating data file #9
Concatenating data file #10
Concatenating data file #11
Concatenating data file #12
Concatenating data file #13
Concatenating data file #14


## Remove front and back matter

In [3]:
reviews = reviews.reset_index(drop = True)
filler = re.compile(r"^[\s\w]*Enjoyment[\s\d]*|\s*Helpful\s*$")
reviews["text"] = reviews["text"].str.replace(filler, "")

## Replace emoticons

In [4]:
# http://introtopython.org/dictionaries.html#General-Syntax
emoticons = {}
for symbol, meaning in EMOTICONS_EMO.items():
    emoticons[symbol] = "_".join(meaning.lower().replace(",", "").split())

In [5]:
# https://www.geeksforgeeks.org/how-to-display-all-rows-from-dataframe-using-pandas/
pd.set_option("display.max_rows", None)
# https://www.pythonforbeginners.com/basics/list-comprehensions-in-python
# https://www.geeksforgeeks.org/python-accessing-key-value-in-dictionary/
# https://stackoverflow.com/questions/4202538/escape-special-characters-in-a-python-string
pattern = "(" + "|".join([re.escape(" " + emoticon) for emoticon in emoticons]) + ")"
used_emoticons = reviews["text"].str.extractall(pattern)

In [10]:
# https://www.digitalocean.com/community/tutorials/python-convert-numpy-array-to-list
used_emoticons = used_emoticons.dropna().drop_duplicates().values.reshape(1, -1)[0].tolist()
# https://stackoverflow.com/questions/5352546/extract-subset-of-key-value-pairs-from-dictionary
used_emoticons = {symbol.lstrip(): emoticons[symbol.lstrip()] for symbol in used_emoticons}
for symbol, meaning in used_emoticons.items():
    print("Replacing " + symbol + " with " + meaning)
    reviews["text"] = reviews["text"].str.replace(" " + symbol, " " + meaning, regex = False)

Replacing :) with happy_face_or_smiley
Replacing :D with laughing_big_grin_or_laugh_with_glasses
Replacing D: with sadness
Replacing :( with frown_sad_andry_or_pouting
Replacing ;) with wink_or_smirk
Replacing :P with tongue_sticking_out_cheeky_playful_or_blowing_a_raspberry
Replacing :3 with happy_face_smiley
Replacing :L with skeptical_annoyed_undecided_uneasy_or_hesitant
Replacing XD with laughing_big_grin_or_laugh_with_glasses
Replacing :/ with skeptical_annoyed_undecided_uneasy_or_hesitant
Replacing :] with happy_face_or_smiley
Replacing ^_^ with joyful
Replacing =) with happy_face_smiley
Replacing :> with happy_face_smiley
Replacing ;_; with sad_or_crying
Replacing :O with surprise
Replacing :} with happy_face_smiley
Replacing :o with surprise
Replacing :S with skeptical_annoyed_undecided_uneasy_or_hesitant
Replacing ;D with wink_or_smirk
Replacing =/ with skeptical_annoyed_undecided_uneasy_or_hesitant
Replacing T.T with sad_or_crying
Replacing =D with laughing_big_grin_or_laugh_

## Replace emojis

In [11]:
emojis = {}
for meaning, symbol in EMOJI_UNICODE.items():
    emojis[symbol] = "_".join(meaning.lower().replace(",", "").replace(":", "").split())

In [12]:
pattern = "(" + "|".join([re.escape(" " + emoji) for emoji in emojis]) + ")"
used_emojis = reviews["text"].str.extractall(pattern)

In [13]:
used_emojis

0
       match        
380    0           😀
591    0           ♥
615    0           ♥
       1           ♥
699    0           😂
       1           😎
1564   0           😀
1924   0           ♥
       1           ♥
2000   0           😊
2193   0           ♥
       1           ♥
2500   0           👍
2502   0           ❤
       1           ❤
       2           ❤
       3           🎉
       4           ❤
2757   0           ♥
3053   0           ♥
3095   0           😊
3156   0           💙
3728   0           ✔
       1           ✔
       2           ✔
       3           ✔
       4           ✔
       5           ✔
       6           ✔
5541   0           ©
5751   0           ♥
5816   0           😊
6135   0           ♥
6215   0           🤷
       1           👐
       2           👐
       3           👐
       4           👐
6514   0           💕
6776   0           😂
7084   0           😂
       1           😎
7250   0           🚨
7518   0           ♥
8050   0           ❤
8375   0           ♥
8461   0           😊
8898   0           🤑
9044   0           💙
9519   0           👏
10045  0           😭
10642  0           📜
       1           👦
       2           💻
       3           📌
11297  0           😓
11819  0           🍟
12910  0           😂
13494  0           👏
       1           👏
       2           👏
       3           👏
       4           👏
       5           👏
       6           👏
       7           👏
       8           👏
       9           👏
       10          👏
       11          👏
       12          👏
13657  0           ♥
       1           ♥
13870  0           👌
       1           🙄
       2           👌
14043  0           ♥
14093  0           🙂
14253  0           ♥
14319  0           😁
14616  0           ®
16568  0           🎼
       1           💘
16762  0           😍
17072  0           🤣
17257  0           ❤
       1           ❤
       2           ❤
       3           ❤
       4           ❤
       5           ❤
       6           ❤
       7           ❤
17380  0           👀
       1           😂
       2           😂
       3           ❤
       4           😂
       5           ❤
17672  0           ™
18041  0           ♥
18098  0           😊
18125  0           ❤
18176  0           😃
       1           😂
       2           😂
18342  0           ♥
18388  0           🙂
18506  0           😎
18910  0           👌
19312  0           🔥
19364  0           👆
19422  0           😅
19454  0           😄
19803  0           😔
19832  0           ❤
       1           ❤
       2           💙
20120  0           ♥
20169  0           😭
       1           😂
20531  0           😂
20983  0           😌
21174  0           ©
21736  0           💙
       1           💙
       2           💛
21974  0           ♥
21992  0           😂
22635  0           😍
23332  0           ♥
23446  0           ✔
       1           ✔
       2           ✔
23626  0           😘
       1           😁
       2           😉
24368  0           🤔
24714  0           ♥
       1           ♥
       2           ♥
       3           ♥
       4           ♥
       5           ♥
       6           ♥
       7           ♥
       8           ♥
       9           ♥
       10          ♥
       11          ♥
       12          ♥
       13          ♥
       14          ♥
       15          ♥
       16          ♥
       17          ♥
       18          ♥
       19          ♥
       20          ♥
       21          ♥
       22          ♥
       23          ♥
       24          ♥
       25          ♥
       26          ♥
       27          ♥
       28          ♥
       29          ♥
       30          ♥
24822  0           ♥
24977  0           😊
25200  0           ❤
26227  0           ♥
28119  0           👌
28259  0           ♥
28332  0           😆
28459  0           😆
       1           😆
30014  0           😢
30051  0           ❤
30752  0           ♥
30951  0           😉
       1           😊
31220  0           😂
31513  0           👌
       1           👌
       2           👌
       3           💯
31793  0           😎
31849  0

In [ ]:
final_characters = reviews["text"].str.extract(r"(?P<final_character>.$)", expand = False)
reviews["text"][final_characters == '❤️']

In [17]:
print(reviews["text"].iloc[17672])

If I were to tell you that an anime called "AssClass" was about a tentacle monster that shows up to a middle school and... stopped right there- you'd think that this was some nasty hentai, right? 
 
Well, let me be the first to say that this show surprised me in several ways.  
No, it doesn't have surprise sex scenes. 
 
Assassination Classroom, where a group of misfit students band together to try and take down their anthropomorphic, superpowered octopus teacher. The only problem is that while he can break all the rules of physics, they can't. How do you kill a monster who threatens to destroy the Earth, and backs his promise up by obliterating the moon, Death Star style?  
 
 
Artwork and Animation: 7 
 
AssClass, as a comedy, has a fairly typical 'cutesy' exaggerated feel to the artwork. Everything is a little larger than life, and some characters are little more than animated caricatures, but it goes along well with the overall tone of the series. It's not made to look hyper-realis

In [18]:
print(reviews["link"].iloc[17672])

https://myanimelist.net/reviews.php?id=190667


In [ ]:
pd.set_option("display.max_rows", None)
final_characters = reviews["text"].str.extract(r"(?P<final_character>.$)")
counts = final_characters.groupby("final_character")["final_character"].count()

In [ ]:
list(counts.index)

In [ ]:
counts[counts.index == '️']
# '⠀', '️', '̿'

## Tokenize text

In [ ]:
reviews["tokenized_text"] = reviews["text"].str.strip().str.lower().str.split(pat = r"[.,]*\s+")

## Create target field

In [ ]:
["cry", "cried", "crying", "sob", "sobbed", "sobbing", "bawl", "bawled", "bawling", "tear", "tears", 😭]

In [ ]:
cry = reviews[["anime_uid", "text"]][reviews["tokenized_text"].apply(lambda x: sum([y == "cry" for y in x])) > 0]

In [ ]:
cry2 = cry.groupby("anime_uid")["anime_uid"].count()

In [ ]:
cry2.plot(kind = "hist", bins = 100)